# aln-enrichment genome prep
### Jaffe, Fuster et al. 2021

In [162]:
import re, glob, os, math, re, scipy, ecopy, gzip, pysam, random
from collections import defaultdict
from ete3 import Tree
import numpy as np
import pandas as pd
from Bio import SeqIO, SearchIO
import matplotlib.pyplot as plt
import subprocess as sp
import seaborn as sns
sns.set('notebook')
%matplotlib inline 
import warnings
warnings.filterwarnings('ignore')

import urllib
from Bio import Entrez
from multiprocessing import Pool
from bs4 import BeautifulSoup
Entrez.email = "alexander_jaffe@berkeley.edu"

In [522]:
def cmdir(path):
    if not os.path.isdir(path):
        os.mkdir(path)
        
def scaffold(gene):
    if gene != "None":
        try: return re.search("(.+?)_[0-9]+$", gene).group(1)
        except: print(gene)

In [3]:
rootdir = "/groups/banfield/projects/environmental/LacPavin/analysis/aln/"
cmdir(rootdir)

# create bins

In [ ]:
cmdir(rootdir + "dna")
cmdir(rootdir + "dna/mapping")
assembly_path = glob.glob(rootdir.replace("analysis", "megahit") + "/*min1000.fa")[0]
fwd_path = "/groups/banfield/sequences/2020/LacPavin_0920_ALND/raw.d/LacPavin_0920_ALND_trim_clean.PE.1.fastq.gz"

### mapping

In [ ]:
# generate mapping
build = "bowtie2-build " + assembly_path + " " + rootdir + "dna/" + os.path.basename(assembly_path)
mapping = "/shared/software/bin/bowtie2 -p 48 -x " + rootdir + "dna/mapping/" + os.path.basename(assembly_path) + " -1 " + \
    fwd_path + " -2 " + fwd_path.replace(".1.", ".2.") + " 2> " + rootdir + "dna/mapping/mapping.log | /shared/software/bin/samtools view -S -b > " + \
    rootdir + "dna/mapping/mapping.bam"
# generate sorted bam + idx
sort = "/shared/software/bin/samtools sort --threads 45 " + rootdir + "dna/mapping/mapping.bam > " + rootdir + "dna/mapping/mapping.sorted.bam"
index = "/shared/software/bin/samtools index -@ 48 " + rootdir + "dna/mapping/mapping.sorted.bam"
print(index)

### auto binning

In [ ]:
cmdir(rootdir + "dna/binning")

In [ ]:
# generate table
jgi = "/shared/software/bin/jgi_summarize_bam_contig_depths --outputDepth " + rootdir + \
    "dna/binning/mapping.txt " + rootdir + "dna/mapping/mapping.sorted.bam"
print(jgi)

In [ ]:
#metabat
cmdir(rootdir + "dna/binning/metabat")
metabat = "/shared/software/bin/metabat2 -t 48 -m 1000 -i " + assembly_path + " -a " + \
    rootdir + "dna/binning/mapping.txt -o " + rootdir + "dna/binning/metabat/metabat"
print(metabat)

In [ ]:
# rename bins
for bin in glob.glob(rootdir + "dna/binning/metabat/metabat*fa"):
    call = "mv " + bin + " " + bin.replace("metabat.", "LacPavin_0920_ALND.metabat")
    sp.call(call, shell=True)

In [ ]:
# generate scaffold to bin - rename bins
filename = rootdir + "dna/binning/LacPavin_0920_ALND.metabat.scaf2bin.txt"

with open(filename, "w") as outfile:
    for bin in glob.glob(rootdir + "dna/binning/metabat/*fa"):
        for record in SeqIO.parse(open(bin), "fasta"):
            outfile.write(record.description.split(" ")[0] + "\t" + os.path.basename(bin).replace(".fa", "") + "\n")

In [ ]:
# fix ggkbase one
ggin = pd.read_csv(rootdir + "dna/binning/LacPavin_0920_ALND.scaffolds_to_bin.txt", sep="\t")
ggin[~ggin["bin"].str.contains("UNK")][["scaffold_name", "bin"]].to_csv(rootdir + "dna/binning/LacPavin_0920_ALND.ggkbase.scaf2bin.txt", header=False, index=False, sep="\t")

In [ ]:
tables = ",".join(glob.glob(rootdir + "dna/binning/*scaf2bin*"))
labels = ",".join(os.path.basename(path).split(".")[1] 
    for path in glob.glob(rootdir + "dna/binning/*scaf2bin*"))

cmdir(rootdir + "dna/binning/dastool")
dastool = "sbatch -J dast --wrap '/shared/software/bin/DAS_Tool -t 48 -i " + tables + " -l " + \
    labels + " -c " + assembly_path + " -o " + rootdir + "dna/binning/dastool/'"
print(dastool)

In [ ]:
table = pd.read_csv(rootdir + "dna/binning/dastool/_DASTool_scaffolds2bin.txt", sep="\t", header=None, names=["scaffold", "bin"])
table.head()

Follow with manual integration of autobins with manual bins, renaming, etc.

# curate bins

Download scaffold 2 bin file from ggkbase.

### create first pass bin files

In [ ]:
cmdir(rootdir + "dna/genomes")
cmdir(rootdir + "dna/genomes/prelim_genomes")

In [4]:
# read in scaffolds
assembly = {record.description.split(" ")[0]: str(record.seq) for record in
    SeqIO.parse(open("/groups/banfield/projects/environmental/LacPavin/QB3_09_14_20/assembly.d/ALND/megahit/LacPavin_0920_ALND_scaffold_min1000.fa"), "fasta")}

In [ ]:
scaf2bin = pd.read_csv(rootdir + "dna/genomes/LacPavin_0920_ALND.scaffolds_to_bin.tsv", sep="\t")

for bin in scaf2bin["bin"].unique():
    if ("UNK" not in bin) and ("Phage" not in bin) and ("Virus" not in bin) \
        and ("virus" not in bin) and ("Mobile" not in bin):
        with open(rootdir + "dna/genomes/prelim_genomes/" + bin + ".fna", "w") as out:
            for key, row in scaf2bin[scaf2bin["bin"]==bin].iterrows():
                out.write(">" + row["scaffold_name"] + "\n" + assembly[row["scaffold_name"]] + "\n")

### quality assessment - all

In [ ]:
call = "sbatch -J checkm --wrap '/shared/software/bin/checkm lineage_wf -t 48 -x .fna " + \
    rootdir + "dna/genomes/prelim_genomes/ " + rootdir + "dna/genomes/checkm_all/'"
call2 = "checkm qa -t 16 -o 1 -f output_table.txt --tab_table lineage.ms ."
print(call2)

In [ ]:
# read in
checkm_results = pd.read_csv(rootdir + "dna/genomes/checkm_all/output_table.txt", sep="\t")
# remove cpr
checkm_sub = checkm_results[(~checkm_results["Bin Id"].str.contains("Parcubacteria")) & (~checkm_results["Bin Id"].str.contains("Kaiser")) \
               & (~checkm_results["Bin Id"].str.contains("PER-ii")) & (~checkm_results["Bin Id"].str.contains("Nomura"))]
checkm_sub = checkm_sub[["Bin Id", "Completeness", "Contamination"]]
checkm_sub.columns = ["genome", "checkm_completeness", "checkm_contamination"]
checkm_sub.head()

### quality assessment - cpr

In [ ]:
basepath = "/groups/banfield/users/ajaffe/cpr-dpann/nr-set-complete/quality/"
outpath = "/groups/banfield/projects/environmental/LacPavin/analysis/aln/dna/genomes/"
cprdir = "/groups/banfield/projects/environmental/LacPavin/analysis/aln/dna/genomes/prelim_cpr_genomes/"
cmdir(cprdir)

In [ ]:
# separate out CPR
for genome in glob.glob(rootdir + "/dna/genomes/prelim_genomes/*"):
    if ("Parcubacteria" in genome) or ("Nomurabacteria" in genome) \
        or ("Kaiser" in genome) or ("PER-ii" in genome):
        sp.call("cp %s %s" %(genome, cprdir), shell=True)

In [16]:
# run checkm with custom marker set
for hmm_set in ["Pfams", "TIGRFAMs"]:
    
    cmdir(outpath + "checkm_cpr_" + hmm_set)
    call = "/shared/software/bin/checkm analyze -t 16 -x .fna " + basepath + "hmms/" + hmm_set + ".hmm " + \
        cprdir + " " + outpath + "checkm_cpr_" + hmm_set
    call2 = "/shared/software/bin/checkm qa -t 16 -o 1 " + basepath + "hmms/" + hmm_set + ".hmm " + \
        outpath + "checkm_cpr_" + hmm_set + " > " + outpath + "checkm_cpr_" + hmm_set + "/" + hmm_set + ".checkm.summary"
    #print(call)
    #print(call2)

In [20]:
# read in checkm results
qa_results = []

for summary in glob.glob(outpath + "*checkm*s_curated/*summary"):

    name = os.path.basename(summary).split(".")[0]
    # reformat table
    with open(summary + ".cleaned", "w") as outfile:
        for line in open(summary).readlines():
            if ("----" not in line) and ("INFO" not in line):
                outfile.write(line)
    
    # back-caculate con/com
    cleaned_table = pd.read_csv(summary + ".cleaned", sep="\s\s+")
    cleaned_table["genome"] = cleaned_table["Bin Id"].apply(lambda x: x.replace(".contigs",""))
    cleaned_table[name + "_com"] = cleaned_table.apply(lambda x: sum([x[i] for i in ['1', '2', '3', '4', '5+']]), axis=1)
    cleaned_table[name + "_con"] = cleaned_table.apply(lambda x: sum([x[i] for i in ['2', '3', '4', '5+']]), axis=1)
    qa_results.append(cleaned_table[["genome", name + "_com", name + "_con"]])

In [21]:
# merge results
qmerged = pd.merge(qa_results[0], qa_results[1])
qmerged["checkm_completeness"] = qmerged.apply(lambda x: (x["Pfams_com"] + x["TIGRFAMs_com"])/float(43)*100, axis=1)
qmerged["checkm_contamination"] = qmerged.apply(lambda x: (x["Pfams_con"] + x["TIGRFAMs_con"])/float(43)*100, axis=1)
qmerged.head()

,genome,TIGRFAMs_com,TIGRFAMs_con,Pfams_com,Pfams_con,checkm_completeness,checkm_contamination
0,LacPavin_0920_ALND_Parcubacteria_55_23,38,0,4,0,97.674419,0.0
1,LacPavin_0920_ALND_Parcubacteria_54_14,38,0,4,0,97.674419,0.0
2,LacPavin_0920_ALND_Parcubacteria_45_23,38,0,4,0,97.674419,0.0
3,LacPavin_0920_ALND_Parcubacteria_41_22,38,0,4,0,97.674419,0.0
4,LacPavin_0920_ALND_PER-ii_52_24,38,0,4,0,97.674419,0.0


### integrate + drep

In [ ]:
cmdir(rootdir + "dna/genomes/dRep-workspace")

In [ ]:
# combine and make quality file
combined = pd.concat([qmerged[["genome", "checkm_completeness", "checkm_contamination"]], checkm_sub])
# write out quality file
with open(rootdir + "dna/genomes/dRep-workspace/genomeInformation.csv", "w") as genome_info:
    genome_info.write("genome,completeness,contamination\n")
    for key, row in combined.iterrows():
        genome_info.write(row["genome"] + ".fna," + str(row["checkm_completeness"]) + "," + str(row["checkm_contamination"]) + "\n")

In [ ]:
def start_dRep():
    
    call = "dRep dereplicate " + rootdir + "dna/genomes/dRep-workspace/ -sa 0.95 -p 20 -comp 70 -con 10 -d -g " + \
        rootdir + "dna/genomes/prelim_genomes/*fna --genomeInfo " + rootdir + "dna/genomes/dRep-workspace/genomeInformation.csv"
    print(call)
    
start_dRep()

Then manually curate the remainders...

# finalize

In [ ]:
cmdir(rootdir + "dna/genomes/final_genomes")

In [6]:
scaf2bin = pd.read_csv(rootdir + "dna/genomes/LacPavin_0920_ALND.scaffolds_to_bin_curated.tsv", sep="\t")
drep_genomes = [os.path.basename(item).split(".")[0] for item in glob.glob(rootdir + "dna/genomes/dRep-workspace/dereplicated_genomes/*")]

In [ ]:
for bin in scaf2bin["bin"].unique():
    if ("UNK" not in bin) and ("Phage" not in bin) and ("Virus" not in bin) \
        and ("virus" not in bin) and ("Mobile" not in bin) and (bin in drep_genomes):
        with open(rootdir + "dna/genomes/final_genomes/" + bin + ".fna", "w") as out:
            for key, row in scaf2bin[scaf2bin["bin"]==bin].iterrows():
                out.write(">" + row["scaffold_name"] + "\n" + assembly[row["scaffold_name"]] + "\n")

In [ ]:
# regenerate quality
call = "sbatch -J checkm --wrap '/shared/software/bin/checkm lineage_wf -t 20 -x .fna " + \
    rootdir + "dna/genomes/final_genomes/ " + rootdir + "dna/genomes/checkm_all_curated/'"
call2 = "checkm qa -t 16 -o 1 -f output_table.txt --tab_table lineage.ms ."
print(call2)

In [7]:
# read in
checkm_results = pd.read_csv(rootdir + "dna/genomes/checkm_all_curated/output_table.txt", sep="\t")
# remove cpr
checkm_sub = checkm_results[(~checkm_results["Bin Id"].str.contains("Parcubacteria")) & (~checkm_results["Bin Id"].str.contains("Kaiser")) \
               & (~checkm_results["Bin Id"].str.contains("PER-ii")) & (~checkm_results["Bin Id"].str.contains("Nomura"))]
checkm_sub = checkm_sub[["Bin Id", "Completeness", "Contamination"]]
checkm_sub.columns = ["genome", "checkm_completeness", "checkm_contamination"]
checkm_sub.head()

,genome,checkm_completeness,checkm_contamination
0,LacPavin_0920_ALND_Actinobacteria_63_63,93.59,2.14
1,LacPavin_0920_ALND_Alphaproteobacteria_58_11,96.16,2.20
2,LacPavin_0920_ALND_Alphaproteobacteria_61_16,76.93,2.33
3,LacPavin_0920_ALND_Armatimonadetes_54_8,93.98,2.78
4,LacPavin_0920_ALND_Bacteria_57_7,73.03,1.78


For CPR, modify commands above...

In [ ]:
cprdir = "/groups/banfield/projects/environmental/LacPavin/analysis/aln/dna/genomes/final_cpr_genomes/"
cmdir(cprdir)

In [ ]:
# separate out CPR
for genome in glob.glob(rootdir + "/dna/genomes/final_genomes/*"):
    if ("Parcubacteria" in genome) or ("Nomurabacteria" in genome) \
        or ("Kaiser" in genome) or ("PER-ii" in genome):
        sp.call("cp %s %s" %(genome, cprdir), shell=True)

Replace Per-ii genome with curated version both here and in final_genomes folder.

In [22]:
# merge results
qmerged = pd.merge(qa_results[0], qa_results[1])
qmerged["checkm_completeness"] = qmerged.apply(lambda x: (x["Pfams_com"] + x["TIGRFAMs_com"])/float(43)*100, axis=1)
qmerged["checkm_contamination"] = qmerged.apply(lambda x: (x["Pfams_con"] + x["TIGRFAMs_con"])/float(43)*100, axis=1)
qmerged.head()

,genome,TIGRFAMs_com,TIGRFAMs_con,Pfams_com,Pfams_con,checkm_completeness,checkm_contamination
0,LacPavin_0920_ALND_Parcubacteria_55_23,38,0,4,0,97.674419,0.0
1,LacPavin_0920_ALND_Parcubacteria_54_14,38,0,4,0,97.674419,0.0
2,LacPavin_0920_ALND_Parcubacteria_45_23,38,0,4,0,97.674419,0.0
3,LacPavin_0920_ALND_Parcubacteria_41_22,38,0,4,0,97.674419,0.0
4,LacPavin_0920_ALND_PER-ii_52_24,38,0,4,0,97.674419,0.0


In [ ]:
# create metadata base
final_metadata = pd.concat([checkm_sub, qmerged[["genome", "checkm_completeness", "checkm_contamination"]]])

scaf_nums = {}
for genome in glob.glob(rootdir + "dna/genomes/final_genomes/*"):
    scaf_nums[os.path.basename(genome).split(".")[0]] = len([record for record in SeqIO.parse(open(genome), "fasta")])
final_metadata["scaffold_number"] = final_metadata["genome"].map(scaf_nums)
final_metadata.head()

,genome,checkm_completeness,checkm_contamination,scaffold_number
0,LacPavin_0920_ALND_Actinobacteria_63_63,93.59,2.14,3162
1,LacPavin_0920_ALND_Alphaproteobacteria_58_11,96.16,2.20,87
2,LacPavin_0920_ALND_Alphaproteobacteria_61_16,76.93,2.33,3803
3,LacPavin_0920_ALND_Armatimonadetes_54_8,93.98,2.78,282
4,LacPavin_0920_ALND_Bacteria_57_7,73.03,1.78,2580


Then write out to manually fill in taxonomy, eventually add NCBI info...

In [26]:
cmdir(rootdir + "tables")
final_metadata.sort_values("genome").to_csv(rootdir + "tables/genome_metadata_base.tsv", sep="\t", index=False)